In [ ]:
# !pip install xlsxwriter
import pandas as pd
import numpy as np
import glob
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.options.display.max_rows = 999
pd.set_option('display.max_columns', None)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def autosize_excel_columns(worksheet, df):
  autosize_excel_columns_df(worksheet, df.index.to_frame())
  autosize_excel_columns_df(worksheet, df, offset=df.index.nlevels)

def autosize_excel_columns_df(worksheet, df, offset=0):
  for idx, col in enumerate(df):
    series = df[col]
    max_len = max((
      series.astype(str).map(len).max(),
      len(str(series.name))
    )) + 1
    worksheet.set_column(idx+offset, idx+offset, max_len)

In [ ]:
xlsx_list = glob.glob("/content/drive/MyDrive/Colab Notebooks/input/ЭКСМО/*.xlsx")
output = pd.DataFrame()
for name in sorted(xlsx_list):
    df = pd.read_excel(name, sheet_name = 'сводная таблица',
                       index_col = [0, 1],
                       usecols = [0, 1, 2, 3, 4])
    sums = df.groupby(level = 0).sum().drop(index='Все', columns='Процент неликвидов')
    df.drop(index='Все', columns = ['Всего новинок', 'Всего неликвидов'], inplace=True)
    sums['ИТОГ % НЕЛЕКВИДОВ'] = sums['Всего неликвидов'] / sums['Всего новинок'] * 100
    sums.rename(columns={'Всего новинок':'ИТОГ ШТ. НОВИНОК', 'Всего неликвидов':'ИТОГ ШТ. НЕЛИКВИДОВ'}, inplace=True)
    sums = sums.stack().to_frame(name='Процент неликвидов')
    sums['dummy'] = 1
    period = pd.concat([df, sums])
    period.rename(columns={'Процент неликвидов':name[-15:-8]}, inplace=True)
    output = pd.concat([output, period], axis=1)
output['sorter'] = output['dummy'].sum(axis=1)
output = output.sort_values(by=['Редакция', 'sorter']).drop(columns=['dummy', 'sorter'])

In [ ]:
writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter') 
output.to_excel(writer, sheet_name='сводная таблица', na_rep='')
worksheet = writer.sheets['сводная таблица']
autosize_excel_columns(worksheet, output)
writer.save()